In [1]:
#Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

#Read in csv file and create dataframe
csv_file = "../Resources/winemag-data-130k-v3.csv"
wine_csv_df = pd.read_csv(csv_file,encoding = "UTF-8")

#Drop region_2 column
wine_csv_df = wine_csv_df.drop(columns="region_2")

#Read in json file and create dataframe
json_file ="../Resources/winemag-data-130k-v3.json"
wine_json_df = pd.read_json(json_file)
wine_json_df.head()

#Merge csv and json dataframes
complete_wine_df = pd.merge(wine_json_df, wine_csv_df, on = "title")

#Drop duplicates, because we hadnt removed duplicates from json
complete_wine_df.drop_duplicates(subset = "title", inplace = True)

#Drop rows containing null values
complete_wine_df = complete_wine_df.dropna()

#Drop duplicates, because we hadnt removed duplicates from json
complete_wine_df.drop_duplicates(subset = "title", inplace = True)

#Reset index so that we have an "index" column that we can manipulate numerically
complete_wine_df = complete_wine_df.reset_index(drop=True).reset_index()

#Add one to each value in the "index" column (indices must start at 1 in pgAdmin)
complete_wine_df["index"] += 1



complete_wine_df.head()

,index,title,description,designation,points,price,taster_name,taster_twitter_handle,country,province,region_1,variety,winery
0,1,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Cabernet Sauvignon,Kirkland Signature
1,2,M. Chapoutier 1999 Le Méal Ermitage (Hermitage),Chapoutier's selections of the best parcels of...,Le Méal Ermitage,98,150.0,Roger Voss,@vossroger,France,Rhône Valley,Hermitage,Rhône-style Red Blend,M. Chapoutier
2,3,E. Guigal 1998 La Mouline (Côte Rôtie),Even though the 42 months this wine has spent ...,La Mouline,96,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal
3,4,E. Guigal 1998 La Turque (Côte Rôtie),"A firm, dry, foursquare wine that has power an...",La Turque,95,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal
4,5,Domaine Jean-Michel Gerin 1999 La Landonne (C...,Here is a superbly rich and smooth wine. Aroma...,La Landonne,95,150.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,Domaine Jean-Michel Gerin


In [2]:
rds_connection_string = 'postgres:Rodri3425!@localhost:5432/wine_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
complete_wine_df.to_sql(name = "complete_wine", con = engine, if_exists = "append",index = False)